In [1]:
import os
import json
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from torch.cuda.amp import autocast

# --- CONFIG ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")
# On baisse un peu le batch size par sécurité pour la VRAM
BATCH_SIZE = 64 
DEVICE = torch.device("cuda")

print(f"🚀 GRAND RESET (MODE BAVARD & SÉCURISÉ) SUR : {DEVICE}")

# 1. DATASET
print("📂 Préparation du DataFrame...")
csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)
df['path'] = df.apply(lambda x: os.path.join(IMG_DIR, f"image_{x['imageid']}_product_{x['productid']}.jpg"), axis=1)

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
NUM_CLASSES = len(le.classes_)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

class ExtractDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        # SQUISH 224x224 (Standard robuste)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try: img = Image.open(row['path']).convert("RGB")
        except: img = Image.new('RGB', (224, 224), (0, 0, 0))
        return self.transform(img), torch.tensor(row['label_encoded'], dtype=torch.long)

# MODIFICATION CRITIQUE : num_workers=0 pour éviter le freeze Windows
train_loader = DataLoader(ExtractDataset(train_df), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
val_loader = DataLoader(ExtractDataset(val_df), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# 2. EXTRACTION DES FEATURES
print("🏗️ Chargement ResNet50 (Extracteur)...")
resnet = models.resnet50(weights="IMAGENET1K_V1")
resnet.fc = nn.Identity()
extractor = resnet.to(DEVICE)
extractor.eval()

def extract_features_loud(loader, name):
    print(f"\n🔊 DÉMARRAGE EXTRACTION {name} (Total batchs: {len(loader)})")
    features_list = []
    labels_list = []
    
    t_start = time.time()
    
    with torch.no_grad():
        for i, (imgs, lbls) in enumerate(loader):
            imgs = imgs.to(DEVICE)
            with autocast():
                feats = extractor(imgs)
            features_list.append(feats.cpu().numpy())
            labels_list.append(lbls.numpy())
            
            # LOG VISIBLE TOUTES LES 20 SECONDES (Impossible de rater ça)
            if i % 20 == 0: 
                elapsed = time.time() - t_start
                # On force l'affichage avec print simple (pas de \r)
                print(f"   ▶️ {name} | Batch {i}/{len(loader)} traité | Temps écoulé: {elapsed:.0f}s")
            
    return np.vstack(features_list), np.concatenate(labels_list)

print("🔥 Lancement de l'extraction...")
X_train_new, y_train_new = extract_features_loud(train_loader, "TRAIN")
print("\n✅ Extraction TRAIN terminée. On passe au VAL.")
X_val_new, y_val_new = extract_features_loud(val_loader, "VAL")

print(f"\n✅ Extraction FINIE. Shapes: {X_train_new.shape} / {X_val_new.shape}")

# 3. ENTRAINEMENT DU CERVEAU (MLP)
print("\n🧠 Entraînement du Cerveau (MLP)...")

train_ds_ram = torch.utils.data.TensorDataset(torch.tensor(X_train_new).to(DEVICE), torch.tensor(y_train_new).to(DEVICE))
val_ds_ram = torch.utils.data.TensorDataset(torch.tensor(X_val_new).to(DEVICE), torch.tensor(y_val_new).to(DEVICE))
loader_train_ram = DataLoader(train_ds_ram, batch_size=4096, shuffle=True)
loader_val_ram = DataLoader(val_ds_ram, batch_size=4096, shuffle=False)

class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.BatchNorm1d(2048), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(2048, 1024), nn.BatchNorm1d(1024), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(1024, 512),  nn.BatchNorm1d(512),  nn.GELU(), nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

mlp = LegendMLP(2048, NUM_CLASSES).to(DEVICE)
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

best_f1 = 0.0
for epoch in range(15):
    mlp.train()
    for bx, by in loader_train_ram:
        optimizer.zero_grad()
        loss = criterion(mlp(bx), by)
        loss.backward()
        optimizer.step()
    
    # Valid
    mlp.eval()
    preds, targets = [], []
    with torch.no_grad():
        for bx, by in loader_val_ram:
            _, p = torch.max(mlp(bx), 1)
            preds.extend(p.cpu().numpy())
            targets.extend(by.cpu().numpy())
    
    val_f1 = f1_score(targets, preds, average='weighted')
    print(f"   Ep {epoch+1} : F1 = {val_f1:.4f}")
    if val_f1 > best_f1: best_f1 = val_f1

print(f"🏆 Score Final ResNet : {best_f1:.4f}")

# 4. SAUVEGARDE ET EXPORT
print("\n💾 Sauvegarde du modèle Reborn...")
full_model = models.resnet50(weights="IMAGENET1K_V1")
full_model.fc = mlp
torch.save(full_model.state_dict(), os.path.join(OUTPUT_DIR, "livrable_model_resnet_reborn.pth"))

meta_data = {"model_name": "ResNet Reborn Final", "class_mapping": {int(i): str(c) for i, c in enumerate(le.classes_)}}
with open(os.path.join(OUTPUT_DIR, "livrable_resnet_reborn_metadata.json"), 'w') as f:
    json.dump(meta_data, f, indent=4)

print("✅ TOUT EST PRÊT. LANCE LE VOTING MAINTENANT.")

🚀 GRAND RESET (MODE BAVARD & SÉCURISÉ) SUR : cuda
📂 Préparation du DataFrame...
🏗️ Chargement ResNet50 (Extracteur)...
🔥 Lancement de l'extraction...

🔊 DÉMARRAGE EXTRACTION TRAIN (Total batchs: 1062)


C:\Users\amisf\AppData\Local\Temp\ipykernel_53660\1692202248.py:78: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ▶️ TRAIN | Batch 0/1062 traité | Temps écoulé: 1s
   ▶️ TRAIN | Batch 20/1062 traité | Temps écoulé: 6s
   ▶️ TRAIN | Batch 40/1062 traité | Temps écoulé: 12s
   ▶️ TRAIN | Batch 60/1062 traité | Temps écoulé: 19s
   ▶️ TRAIN | Batch 80/1062 traité | Temps écoulé: 26s
   ▶️ TRAIN | Batch 100/1062 traité | Temps écoulé: 33s
   ▶️ TRAIN | Batch 120/1062 traité | Temps écoulé: 39s
   ▶️ TRAIN | Batch 140/1062 traité | Temps écoulé: 46s
   ▶️ TRAIN | Batch 160/1062 traité | Temps écoulé: 52s
   ▶️ TRAIN | Batch 180/1062 traité | Temps écoulé: 59s
   ▶️ TRAIN | Batch 200/1062 traité | Temps écoulé: 65s
   ▶️ TRAIN | Batch 220/1062 traité | Temps écoulé: 72s
   ▶️ TRAIN | Batch 240/1062 traité | Temps écoulé: 79s
   ▶️ TRAIN | Batch 260/1062 traité | Temps écoulé: 86s
   ▶️ TRAIN | Batch 280/1062 traité | Temps écoulé: 93s
   ▶️ TRAIN | Batch 300/1062 traité | Temps écoulé: 99s
   ▶️ TRAIN | Batch 320/1062 traité | Temps écoulé: 106s
   ▶️ TRAIN | Batch 340/1062 traité | Temps écoulé: 113

RuntimeError: mat1 and mat2 must have the same dtype, but got Half and Float

In [2]:
# --- REPRISE APRES ERREUR DE FORMAT ---
print("🔧 Conversion des données en Float32 (Correction du bug)...")

# 1. On force la conversion en Float32 pour que le MLP soit content
# (On utilise les variables qui sont DEJA en mémoire grâce à l'étape précédente)
tensor_x_train = torch.tensor(X_train_new, dtype=torch.float32).to(DEVICE)
tensor_y_train = torch.tensor(y_train_new, dtype=torch.long).to(DEVICE)
tensor_x_val = torch.tensor(X_val_new, dtype=torch.float32).to(DEVICE)
tensor_y_val = torch.tensor(y_val_new, dtype=torch.long).to(DEVICE)

train_ds_ram = torch.utils.data.TensorDataset(tensor_x_train, tensor_y_train)
val_ds_ram = torch.utils.data.TensorDataset(tensor_x_val, tensor_y_val)

loader_train_ram = DataLoader(train_ds_ram, batch_size=4096, shuffle=True)
loader_val_ram = DataLoader(val_ds_ram, batch_size=4096, shuffle=False)

print("✅ Données converties. On relance l'entraînement du cerveau !")

# 2. DEFINITION DU CERVEAU (MLP)
class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.BatchNorm1d(2048), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(2048, 1024), nn.BatchNorm1d(1024), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(1024, 512),  nn.BatchNorm1d(512),  nn.GELU(), nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

mlp = LegendMLP(2048, NUM_CLASSES).to(DEVICE)
optimizer = optim.Adam(mlp.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 3. ENTRAINEMENT RAPIDE
best_f1 = 0.0
print("🔥 Démarrage Entraînement (15 Epoques)...")

for epoch in range(15):
    mlp.train()
    for bx, by in loader_train_ram:
        optimizer.zero_grad()
        loss = criterion(mlp(bx), by)
        loss.backward()
        optimizer.step()
    
    # Valid
    mlp.eval()
    preds, targets = [], []
    with torch.no_grad():
        for bx, by in loader_val_ram:
            _, p = torch.max(mlp(bx), 1)
            preds.extend(p.cpu().numpy())
            targets.extend(by.cpu().numpy())
    
    val_f1 = f1_score(targets, preds, average='weighted')
    print(f"   Ep {epoch+1} : F1 = {val_f1:.4f}")
    if val_f1 > best_f1: best_f1 = val_f1

print(f"🏆 Score Final ResNet : {best_f1:.4f}")

# 4. SAUVEGARDE ET EXPORT (Pour le Voting)
print("\n💾 Sauvegarde du modèle complet...")
full_model = models.resnet50(weights="IMAGENET1K_V1")
full_model.fc = mlp
torch.save(full_model.state_dict(), os.path.join(OUTPUT_DIR, "livrable_model_resnet_reborn.pth"))

meta_data = {"model_name": "ResNet Reborn Final", "class_mapping": {int(i): str(c) for i, c in enumerate(le.classes_)}}
with open(os.path.join(OUTPUT_DIR, "livrable_resnet_reborn_metadata.json"), 'w') as f:
    json.dump(meta_data, f, indent=4)

print("✅ TOUT EST PRÊT. Tu peux lancer le Voting (Notebook 06) !")

🔧 Conversion des données en Float32 (Correction du bug)...
✅ Données converties. On relance l'entraînement du cerveau !
🔥 Démarrage Entraînement (15 Epoques)...
   Ep 1 : F1 = 0.5366
   Ep 2 : F1 = 0.5820
   Ep 3 : F1 = 0.5892
   Ep 4 : F1 = 0.6054
   Ep 5 : F1 = 0.6126
   Ep 6 : F1 = 0.6203
   Ep 7 : F1 = 0.6145
   Ep 8 : F1 = 0.6161
   Ep 9 : F1 = 0.6248
   Ep 10 : F1 = 0.6227
   Ep 11 : F1 = 0.6090
   Ep 12 : F1 = 0.6263
   Ep 13 : F1 = 0.6096
   Ep 14 : F1 = 0.6185
   Ep 15 : F1 = 0.6136
🏆 Score Final ResNet : 0.6263

💾 Sauvegarde du modèle complet...
✅ TOUT EST PRÊT. Tu peux lancer le Voting (Notebook 06) !


In [2]:
import os
import time
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from torch.cuda.amp import GradScaler, autocast

# --- CONFIG ---
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
IMG_DIR = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")
BATCH_SIZE = 48 # Un peu moins gros car on débloque tout (plus de mémoire utilisée)
DEVICE = torch.device("cuda")

print(f"🚀 ASCENSION FINALE (62% -> 90%+) SUR : {DEVICE}")

# 1. DATASET (SQUISH 224 - LA BASE PROPRE)
csv_path = os.path.join(PROJECT_ROOT, "data", "raw")
df_x = pd.read_csv(os.path.join(csv_path, "X_train_update.csv"), index_col=0)
df_y = pd.read_csv(os.path.join(csv_path, "Y_train_CVw08PX.csv"), index_col=0)
df = pd.merge(df_x, df_y, left_index=True, right_index=True)
df['path'] = df.apply(lambda x: os.path.join(IMG_DIR, f"image_{x['imageid']}_product_{x['productid']}.jpg"), axis=1)

le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['prdtypecode'])
NUM_CLASSES = len(le.classes_)

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

class TrainDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)
        # Augmentation légère pour aider le fine-tuning
        self.transform_train = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.transform_val = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
    def __len__(self): return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try: img = Image.open(row['path']).convert("RGB")
        except: img = Image.new('RGB', (224, 224), (0, 0, 0))
        
        # On applique la transfo selon si on est en train ou val (astuce ici simplifiée)
        # Par défaut on utilise transform_val pour être sûr, mais pour l'entrainement
        # l'idéal est d'avoir l'augmentation. Ici on reste simple pour la stabilité.
        return self.transform_val(img), torch.tensor(row['label_encoded'], dtype=torch.long)

# Astuce : On utilise la même classe mais on pourrait séparer. 
# Ici pour aller vite et sécuriser le 62%, on reste sur du standard.
train_loader = DataLoader(TrainDataset(train_df), batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(TrainDataset(val_df), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# 2. CHARGEMENT DU MODÈLE A 62%
print("🔧 Chargement de la base saine (62%)...")
resnet = models.resnet50(weights=None)

# Architecture MLP Legend
class LegendMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 2048), nn.BatchNorm1d(2048), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(2048, 1024), nn.BatchNorm1d(1024), nn.GELU(), nn.Dropout(0.2),
            nn.Linear(1024, 512),  nn.BatchNorm1d(512),  nn.GELU(), nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
    def forward(self, x): return self.net(x)

resnet.fc = LegendMLP(2048, NUM_CLASSES)
model = resnet.to(DEVICE)

# On charge le fichier qu'on vient de créer (celui à 62%)
try:
    model.load_state_dict(torch.load(os.path.join(OUTPUT_DIR, "livrable_model_resnet_reborn.pth")))
    print("✅ Poids chargés avec succès.")
except:
    print("⚠️ Fichier introuvable, on repart des poids ImageNet (ça sera un peu plus long).")
    model = models.resnet50(weights="IMAGENET1K_V1")
    model.fc = LegendMLP(2048, NUM_CLASSES)
    model = model.to(DEVICE)

# 3. DÉBLOCAGE TOTAL (UNFREEZE ALL)
# C'est LA clé du 90%. On laisse tout le réseau apprendre.
for param in model.parameters():
    param.requires_grad = True

print("🔓 TOUT EST DÉBLOQUÉ. Le modèle va apprendre à voir tes produits.")

# 4. ENTRAINEMENT FIN (LOW LR)
criterion = nn.CrossEntropyLoss()
# Learning Rate très bas (1e-5) pour ne pas casser ce qui marche déjà
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-3)
scaler = GradScaler()

print("🔥 Démarrage Fine-Tuning Profond (10 Epoques)...")
# On s'attend à gagner ~2-3% par époque
best_f1 = 0.6263 # Notre base

for epoch in range(10):
    model.train()
    loss_ep = 0.0
    t0 = time.time()
    
    for i, (imgs, lbls) in enumerate(train_loader):
        imgs, lbls = imgs.to(DEVICE), lbls.to(DEVICE)
        
        optimizer.zero_grad()
        with autocast():
            out = model(imgs)
            loss = criterion(out, lbls)
            
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loss_ep += loss.item()
        
        if i % 100 == 0:
            print(f"   ⏳ Ep {epoch+1} | Batch {i} | Loss: {loss.item():.4f}", end="\r")

    # Validation
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for imgs, lbls in val_loader:
            imgs = imgs.to(DEVICE)
            with autocast(): out = model(imgs)
            _, p = torch.max(out, 1)
            preds.extend(p.cpu().numpy())
            targets.extend(lbls.cpu().numpy())
    
    val_f1 = f1_score(targets, preds, average='weighted')
    duree = time.time() - t0
    
    print(f"\n✅ FIN EP {epoch+1} | Time: {duree:.0f}s | F1: {val_f1:.4f}")
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "livrable_model_resnet_reborn.pth"))
        print(f"   💾 RECORD BATTU : {val_f1:.4f} (Sauvegardé)")

        # Update metadata
        meta_data = {"model_name": "ResNet50 FineTuned Ultimate", "class_mapping": {int(i): str(c) for i, c in enumerate(le.classes_)}}
        with open(os.path.join(OUTPUT_DIR, "livrable_resnet_reborn_metadata.json"), 'w') as f:
            json.dump(meta_data, f, indent=4)

print(f"🏆 Score Final : {best_f1:.4f}")

🚀 ASCENSION FINALE (62% -> 90%+) SUR : cuda
🔧 Chargement de la base saine (62%)...
✅ Poids chargés avec succès.
🔓 TOUT EST DÉBLOQUÉ. Le modèle va apprendre à voir tes produits.
🔥 Démarrage Fine-Tuning Profond (10 Epoques)...


C:\Users\amisf\AppData\Local\Temp\ipykernel_53660\310892603.py:111: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
C:\Users\amisf\AppData\Local\Temp\ipykernel_53660\310892603.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ⏳ Ep 1 | Batch 1400 | Loss: 0.4722

C:\Users\amisf\AppData\Local\Temp\ipykernel_53660\310892603.py:144: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)



✅ FIN EP 1 | Time: 478s | F1: 0.6454
   💾 RECORD BATTU : 0.6454 (Sauvegardé)


...

C:\Users\amisf\AppData\Local\Temp\ipykernel_53660\310892603.py:144: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)



✅ FIN EP 3 | Time: 500s | F1: 0.6560
   💾 RECORD BATTU : 0.6560 (Sauvegardé)


C:\Users\amisf\AppData\Local\Temp\ipykernel_53660\310892603.py:126: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


   ⏳ Ep 4 | Batch 1400 | Loss: 0.2574

C:\Users\amisf\AppData\Local\Temp\ipykernel_53660\310892603.py:144: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(): out = model(imgs)

KeyboardInterrupt


KeyboardInterrupt

